In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_parquet('C:\\Users\\uddip\\Downloads\\ABAWS2324.parq')

In [8]:
#to display full dataframe
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# Set the display option to avoid scientific notation
#pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [3]:
filtered_df = df[df['timecode'] == '0']

In [4]:
selected_columns = ['extr_extr01_melt01_pressure_r_value_act', 'extr_extr01_melt01_temp_r_value_act', 'extr_extr01_screw_load_r_value_act', 'extr_extr01_screw_rotspeed_r_value_act']

In [5]:
filtered_df_new = filtered_df[selected_columns]

# stationary check, ADF Testing

In [9]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Define the selected columns
selected_columns = ['extr_extr01_funnel_active_r_value_act', 
                    'extr_extr01_melt01_pressure_r_value_act', 
                    'extr_extr01_melt01_temp_r_value_act', 
                    'extr_extr01_screw_load_r_value_act', 
                    'extr_extr01_vacuum_r_value_act', 
                    'extr_extr01_screw_rotspeed_r_value_act']

# Handle missing values
filtered_df_new.dropna(inplace=True)  # Drop rows with missing values

# Perform ADF test for each column and print results
for column in selected_columns:
    print(f"ADF test results for column: {column}")
    result = adfuller(filtered_df_new[column])
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("Critical Values:")
    for key, value in result[4].items():
        print(f"\t{key}: {value}")
    print("\n")


C:\Users\uddip\AppData\Local\Temp\ipykernel_25928\465753729.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_new.dropna(inplace=True)  # Drop rows with missing values


ADF test results for column: extr_extr01_funnel_active_r_value_act
ADF Statistic: -6.222558890608615
p-value: 5.180752475896608e-08
Critical Values:
	1%: -3.4304356283119124
	5%: -2.8615778462432964
	10%: -2.56679014425346


ADF test results for column: extr_extr01_melt01_pressure_r_value_act
ADF Statistic: -7.549457338974977
p-value: 3.217603578903355e-11
Critical Values:
	1%: -3.4304356462557153
	5%: -2.861577854174034
	10%: -2.566790148474739


ADF test results for column: extr_extr01_melt01_temp_r_value_act
ADF Statistic: -8.334330791650236
p-value: 3.3034844177327675e-13
Critical Values:
	1%: -3.4304355969283433
	5%: -2.8615778323724976
	10%: -2.566790136870476


ADF test results for column: extr_extr01_screw_load_r_value_act
ADF Statistic: -7.172541377339293
p-value: 2.780345725354147e-10
Critical Values:
	1%: -3.4304356484992193
	5%: -2.8615778551656104
	10%: -2.5667901490025233


ADF test results for column: extr_extr01_vacuum_r_value_act
ADF Statistic: -12.172141824926248
p-va

# Granger Test

In [6]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Define the selected columns
selected_columns = ['extr_extr01_melt01_pressure_r_value_act', 
                    'extr_extr01_melt01_temp_r_value_act', 
                    'extr_extr01_screw_load_r_value_act', 
                    'extr_extr01_screw_rotspeed_r_value_act'
                   ]

# Fill missing values using forward fill and backward fill
filtered_df_new.fillna(method='ffill', inplace=True)  # Forward fill missing values
filtered_df_new.fillna(method='bfill', inplace=True)  # Backward fill remaining missing values

#time lag depends on the frequency of the data

# Perform Granger causality test for each pair of columns
for target_column in selected_columns:
    print(f"Granger causality test results for {target_column}:")
    for potential_cause_column in selected_columns:
        if potential_cause_column != target_column:
            print(f"Potential cause: {potential_cause_column}")
            data = pd.concat([filtered_df_new[target_column], filtered_df_new[potential_cause_column]], axis=1)
            result = grangercausalitytests(data, maxlag=3, verbose=False)
            print(result)
            print()


C:\Users\uddip\AppData\Local\Temp\ipykernel_19964\2907356674.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_new.dropna(inplace=True)  # Drop rows with missing values


Granger causality test results for extr_extr01_melt01_pressure_r_value_act:
Potential cause: extr_extr01_melt01_temp_r_value_act
{1: ({'ssr_ftest': (15.481856724841766, 8.337461003967321e-05, 76414.0, 1), 'ssr_chi2test': (15.48246453977325, 8.327422052214234e-05, 1), 'lrtest': (15.480896339402534, 8.334333967211195e-05, 1), 'params_ftest': (15.481856724794994, 8.337461004163988e-05, 76414.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E9148E0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E914AC0>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (7.4260690896550345, 0.000595953762124276, 76411.0, 2), 'ssr_chi2test': (14.853110037954721, 0.0005952345556176336, 2), 'lrtest': (14.851666712726, 0.0005956642691786547, 2), 'params_ftest': (7.426069089660704, 0.0005959537621179745, 76411.0, 2.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E9147F0>, <statsmodels.regression.

{1: ({'ssr_ftest': (23.52089920203622, 1.2380679336936233e-06, 76414.0, 1), 'ssr_chi2test': (23.521822628340377, 1.23504948046156e-06, 1), 'lrtest': (23.518203259271104, 1.2373749909628093e-06, 1), 'params_ftest': (23.520899202611623, 1.2380679333188358e-06, 76414.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E914BE0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E9145E0>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (381.8944080915155, 9.306957574360566e-166, 76411.0, 2), 'ssr_chi2test': (763.8387951661737, 1.363054113823862e-166, 2), 'lrtest': (760.0464572259225, 9.078389636267407e-166, 2), 'params_ftest': (381.894408091369, 9.306957575717551e-166, 76411.0, 2.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E93E0A0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E93E610>, array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0

{1: ({'ssr_ftest': (1.4502671897938277, 0.22848881460351522, 76414.0, 1), 'ssr_chi2test': (1.4503241270248246, 0.22847595071163512, 1), 'lrtest': (1.4503103645402007, 0.2284781584266867, 1), 'params_ftest': (1.4502671902122417, 0.22848881453624284, 76414.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E9143A0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E914640>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (229.71890384988887, 3.4125747361437664e-100, 76411.0, 2), 'ssr_chi2test': (459.46787129060226, 1.6897382818019808e-100, 2), 'lrtest': (458.0920580463717, 3.361812544249224e-100, 2), 'params_ftest': (229.71890385004153, 3.412574735619238e-100, 76411.0, 2.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E93E6A0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E93E7C0>, array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])

{1: ({'ssr_ftest': (3286.0262907086044, 0.0, 76414.0, 1), 'ssr_chi2test': (3286.1552995142174, 0.0, 1), 'lrtest': (3217.4606197801186, 0.0, 1), 'params_ftest': (3286.026290708735, 0.0, 76414.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E914B80>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E914A00>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (1502.6173647320097, 0.0, 76411.0, 2), 'ssr_chi2test': (3005.431378816172, 0.0, 2), 'lrtest': (2947.8350869469577, 0.0, 2), 'params_ftest': (1502.6173647320147, 0.0, 76411.0, 2.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E93E7C0>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001FD5E9487F0>, array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]])]), 3: ({'ssr_ftest': (915.9007814920172, 0.0, 76408.0, 3), 'ssr_chi2test': (2747.9540709498674, 0.0, 3), 'lrtest': (2699.698071073508, 0.0, 3